In [1]:
df_raw = spark.read.options(header='False',wholetext=True)\
    .text('adl://cp-bizops-c15.azuredatalakestore.net/local/users/lowen/UIUC/dblp.xml')
df_raw.show()

StatementMeta(sparkpool3, 26, 3, Finished, Available)

+--------------------+
|               value|
+--------------------+
|<?xml version="1....|
|<!DOCTYPE dblp SY...|
|              <dblp>|
|<phdthesis mdate=...|
|<author>Carmen He...|
|<title>Modell zur...|
|   <year>2010</year>|
|<school>Aarhus Un...|
|<pages>1-315</pages>|
|<isbn>978-3-86596...|
|<ee>http://d-nb.i...|
|</phdthesis><phdt...|
|<author>Gerd Hoff...|
|<title>Ein Verfah...|
|   <year>2002</year>|
|<school>Universit...|
|<ee>http://ubt.op...|
|<ee>https://nbn-r...|
|<ee>http://d-nb.i...|
|        </phdthesis>|
+--------------------+
only showing top 20 rows

In [2]:
df=df_raw.select('value').collect()

StatementMeta(sparkpool3, 26, 4, Finished, Available)

In [3]:
mstr=""
flag=False
dblp=[]
for row in df:
    s=str(row.value)
    if s.__contains__('</phdthesis>'):
        i=s.index('</phdthesis>')
        s1=s[0:i+len('</phdthesis>')]
        mstr=mstr+s1
        dblp.append(mstr)
        mstr=""
        flag=False
    if s.__contains__('<phdthesis'):
        i=s.index('<phdthesis')
        mstr=mstr+s[i:]
        flag=True
    else:
        if flag==True:
            mstr=mstr+s

StatementMeta(sparkpool3, 26, 5, Finished, Available)

In [4]:
import xml.etree.ElementTree as ET 
list_dblp=[]
for d in dblp:
    dict_dblp={}
    try:
        tree = ET.fromstring(d.replace('&','')) 
        for child in tree:
            #print(child.tag, ' '.join(child.itertext()))
            dict_dblp[child.tag]=' '.join(child.itertext())
        list_dblp.append(dict_dblp)
    except :
        print(d)

list_dblp[:2]

StatementMeta(sparkpool3, 26, 6, Finished, Available)

[{'author': 'Carmen Heine', 'title': 'Modell zur Produktion von Online-Hilfen.', 'year': '2010', 'school': 'Aarhus University', 'pages': '1-315', 'isbn': '978-3-86596-263-8', 'ee': 'http://d-nb.info/996064095'}, {'author': 'Gerd Hoff', 'title': 'Ein Verfahren zur thematisch spezialisierten Suche im Web und seine Realisierung im Prototypen HomePageSearch', 'year': '2002', 'school': 'University of Trier, Germany', 'ee': 'http://d-nb.info/971713243'}]

In [13]:
###get the list of author & title
list_school_title=[]
list_school=[]
list_title=[]
for l in list_dblp:
    list_school_title.append([l.get('school'),l.get('title')])
print(list_school_title[:3])

for a in list_school_title:
    list_school.append(a[0])
list_school[:3]

for a in list_school_title:
    list_title.append(a[1])
list_title[:3]



StatementMeta(sparkpool3, 26, 15, Finished, Available)

[['Aarhus University', 'Modell zur Produktion von Online-Hilfen.'], ['University of Trier, Germany', 'Ein Verfahren zur thematisch spezialisierten Suche im Web und seine Realisierung im Prototypen HomePageSearch'], ['University of California at Berkeley', 'File System Performance and Transaction Support.']]
['Modell zur Produktion von Online-Hilfen.', 'Ein Verfahren zur thematisch spezialisierten Suche im Web und seine Realisierung im Prototypen HomePageSearch', 'File System Performance and Transaction Support.']

In [15]:
###get the top author
from collections import Counter
dict_school={}
for auth in list_school:
    dict_school[auth]=dict_school.get(auth,0)+1

dict_school_count=Counter(dict_school)
dict_school_topcount=dict_school_count.most_common(200)

#top_words=[w[0] for w in top_words]
###2. deal with the two words phrase

dict_school_topcount[:50]

StatementMeta(sparkpool3, 26, 17, Finished, Available)

[('Massachusetts Institute of Technology, Cambridge, MA, USA', 2165), ('Technical University Munich, Germany', 1114), ('Georgia Institute of Technology, Atlanta, GA, USA', 1100), ('Dresden University of Technology, Germany', 1039), ('University of Satilde;o Paulo, Brazil', 1012), ('RWTH Aachen University, Germany', 1009), ('Karlsruhe Institute of Technology, Germany', 983), ('Darmstadt University of Technology, Germany', 964), ('Joseph Fourier University, Grenoble, France', 953), ('University of Maryland, College Park, MD, USA', 855), ('Technical University of Berlin, Germany', 842), ('Imperial College London, UK', 744), ('Grenoble Institute of Technology, France', 731), ('University of Campinas, Brazil', 711), ('University of Southampton, UK', 680), ('University of Stuttgart, Germany', 679), ('Grenoble Alpes University, France', 672), ('University of Edinburgh, UK', 660), ('University of Rennes 1, France', 654), ('Technical University Munich', 629), ('University of Erlangen-Nuremberg,

In [20]:
from gensim import corpora
### data cleaning for title
from gensim.utils import simple_preprocess
list_title_clean = [simple_preprocess(doc) for doc in list_title]
##remove words with length<=3
list_title_clean=[[w for w in s if len(w)>3]  for s in list_title_clean ]
##stop words
stopword=['http','https','university', 'info']
list_title_clean=[[w for w in s if w not in stopword]  for s in list_title_clean ]

#dct = corpora.Dictionary(list_title_clean)

#corpus = [dct.doc2bow(line) for line in tokenized_list]
#corpus[:1]

list_title_clean[:3]

StatementMeta(sparkpool3, 26, 22, Finished, Available)

[['modell', 'produktion', 'online', 'hilfen'], ['verfahren', 'thematisch', 'spezialisierten', 'suche', 'seine', 'realisierung', 'prototypen', 'homepagesearch'], ['file', 'system', 'performance', 'transaction', 'support']]

In [24]:
###1. get the dictionary for all data
from collections import Counter
##1.1. get the (word, count) pair dictionary
dictionary={}

for s in list_title_clean:
    for w in s:
        dictionary[w]=dictionary.get(w,0)+1
##1.2. sorting to get the top 500 words
top_words=Counter(dictionary)
top_words=top_words.most_common(200)
#topwords_dict
top_words=[w[0] for w in top_words]
###2. deal with the two words phrase
top_words[:10]

word_dict={}
i=1
for word in top_words:
    word_dict[word]=i
    i=i+1
#print(word_dict.get('germany'))
word_dict
                                                                

StatementMeta(sparkpool3, 26, 26, Finished, Available)

{'systems': 1, 'based': 2, 'pour': 3, 'fuuml': 4, 'agrave': 5, 'data': 6, 'analysis': 7, 'networks': 8, 'with': 9, 'atilde': 10, 'reacute': 11, 'application': 12, 'applications': 13, 'design': 14, 'eacute': 15, 'learning': 16, 'using': 17, 'multi': 18, 'system': 19, 'dans': 20, 'information': 21, 'software': 22, 'deacute': 23, 'model': 24, 'models': 25, 'approach': 26, 'meacute': 27, 'systegrave': 28, 'algorithms': 29, 'analyse': 30, 'simulation': 31, 'from': 32, 'methods': 33, 'time': 34, 'management': 35, 'control': 36, 'modeling': 37, 'distributed': 38, 'eines': 39, 'performance': 40, 'computer': 41, 'image': 42, 'techniques': 43, 'network': 44, 'framework': 45, 'development': 46, 'wireless': 47, 'efficient': 48, 'optimization': 49, 'detection': 50, 'images': 51, 'dynamic': 52, 'processing': 53, 'para': 54, 'mobile': 55, 'recognition': 56, 'entwicklung': 57, 'modegrave': 58, 'modelling': 59, 'lisation': 60, 'high': 61, 'modeacute': 62, 'architecture': 63, 'communication': 64, 'machi

In [31]:
###2.3 output
output_1=[]
output_2=[]
output_shoool=[]
for s in list_title_clean:
    i=list_title_clean.index(s)
    top_words_temp=top_words.copy()
    word_group_1=[]
    word_group_2=[]
    for w in top_words:
        word_first=''
        if w in s:
            word_first=w
            word_group_1.append(str(word_dict.get(w)))
            word_group_1.append('-1')
            word_group_2.append(w)
            #print(word_first)
            if w in top_words_temp:
                top_words_temp.remove(w)
            else:
                print(w)
            for w1 in top_words_temp:
                if w1 in s:
                    word_group_1.append(str(word_dict.get(word_first))+' ' +str(word_dict.get(w1)))
                    word_group_1.append('-1')
                    word_group_2.append(word_first+' ' +w1)
    word_group_1.append('-2')
    #print(word_group)
    l=''
    for s in word_group_1:
        l=l+' '+s
    l=l[1:]
    if l!='-2':
        output_1.append(l)
        output_shoool.append(list_school[i])
       

    if len(word_group_2)!=0:
        output_2.append(word_group_2)


StatementMeta(sparkpool3, 26, 33, Finished, Available)

In [28]:
## output for top schools
list_school_desc=[w[0] for w in dict_school_topcount]
###2. deal with the two words phrase
list_top_shool=list_school_desc[:10]
list_top_shool

StatementMeta(sparkpool3, 26, 30, Finished, Available)

['Massachusetts Institute of Technology, Cambridge, MA, USA', 'Technical University Munich, Germany', 'Georgia Institute of Technology, Atlanta, GA, USA', 'Dresden University of Technology, Germany', 'University of Satilde;o Paulo, Brazil', 'RWTH Aachen University, Germany', 'Karlsruhe Institute of Technology, Germany', 'Darmstadt University of Technology, Germany', 'Joseph Fourier University, Grenoble, France', 'University of Maryland, College Park, MD, USA']

In [43]:
#### output top school
output_title_top=[]
output_school_top=[]
for i in range(len(output_shoool)):
    if output_shoool[i] in list_top_shool:
        output_title_top.append(output_1[i])
        output_school_top.append(output_shoool[i])


StatementMeta(sparkpool3, 26, 45, Finished, Available)

In [44]:
print(output_title_top[:3])
print(output_school_top[:3])

StatementMeta(sparkpool3, 26, 46, Finished, Available)

['2 -1 2 196 -1 196 -1 -2', '13 -1 13 38 -1 13 53 -1 13 54 -1 13 55 -1 13 63 -1 13 161 -1 38 -1 38 53 -1 38 54 -1 38 55 -1 38 63 -1 38 161 -1 53 -1 53 54 -1 53 55 -1 53 63 -1 53 161 -1 54 -1 54 55 -1 54 63 -1 54 161 -1 55 -1 55 63 -1 55 161 -1 63 -1 63 161 -1 161 -1 -2', '94 -1 94 112 -1 112 -1 -2']
['RWTH Aachen University, Germany', 'University of Satilde;o Paulo, Brazil', 'Massachusetts Institute of Technology, Cambridge, MA, USA']

In [68]:
columns = ["value"]

df_title = spark.createDataFrame(data=[[p] for p in output_title_top],schema =columns)
df_title.coalesce(1).write.format("text").option("header", "false").mode("overwrite").save('adl://cp-bizops-c15.azuredatalakestore.net/local/users/lowen/UIUC/dblp_title.txt')

df_school = spark.createDataFrame(data=[[p] for p in output_school_top],schema =columns)
df_school.coalesce(1).write.format("text").option("header", "false").mode("overwrite").save('adl://cp-bizops-c15.azuredatalakestore.net/local/users/lowen/UIUC/dblp_school.txt')


print(df_title.count(),df_school.count())

StatementMeta(sparkpool3, 26, 70, Finished, Available)

9573 9573

In [73]:
import pandas as pd

#df_dict=pd.DataFrame.from_dict(word_dict[0])
list_word_top= [[key +' '+str(value)] for key, value in word_dict.items()]

df_word = spark.createDataFrame(data=list_word_top,schema =columns)
df_word.coalesce(1).write.format("text").option("header", "false").mode("overwrite").save('adl://cp-bizops-c15.azuredatalakestore.net/local/users/lowen/UIUC/dblp_word.txt')
#display(df_word.limit(5))

list_word_top[:3]

StatementMeta(sparkpool3, 26, 75, Finished, Available)

[['systems 1'], ['based 2'], ['pour 3']]